## Segmenting and Clustering Neighborhoods in Toronto part :1

In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup 
import requests 

#### 1-Downloading data and  Creating the pandas data frame :

In [2]:
df = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
sp = BeautifulSoup(df, 'html.parser')


In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:
sp.find('table').find_all('tr')
sp.find('table').find_all('tr')
for row in sp.find('table').find_all('tr'):
    cells = row.find_all('td')
    

In [5]:
for row in sp.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) 

In [6]:
dframe = pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})
dframe.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 2-correcting the  string format  in the Borough column and  Ignoring cells with a borough that is Not assigned:

In [7]:
dframe.drop(dframe.index[dframe['Borough'] == 'Not assigned'], inplace = True)
dframe = dframe.reset_index(drop=True)
dframe.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 3-Grouping Neighbourhoods by similar Postcode and Borough  :

In [8]:
dframe = dframe.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
dframe.columns = ['PostalCode','Borough','Neighborhood']
dframe.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
dframe.shape

(103, 3)